In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("domainadaptationexperiments.csv").dropna()
#df

In [2]:
df_full = df[df['test_region_type'] == 'full'].drop(columns=['test_region_type'])

In [3]:
df_after = df_full[df_full['adaptation_state'] == 'after_adaptation'].drop(columns=['adaptation_state'])

In [4]:
group_columns = ['source_train_dataset', 'target_train_dataset', 'training_method']
avg_columns = ['test_abs_rel', 'test_sq_rel', 'test_log10', 'test_rmse', 'test_rmse_log', 'test_a1', 'test_a2', 'test_a3']
df_stability = df_after.groupby(group_columns, as_index=False)[avg_columns].mean()
#df_stability

In [5]:
df_before = df_full[df_full['adaptation_state'] == 'before_adaptation'].drop(columns=['adaptation_state'])
#df_before

In [6]:
df_before_plasticity = df_before[df_before['test_dataset'] == df_before['source_train_dataset']]
#df_before_plasticity

In [7]:
df_after_plasticity = df_after[df_after['test_dataset'] == df_after['target_train_dataset']]
#df_after_plasticity

In [8]:
df_plasticity = pd.concat([df_before_plasticity, df_after_plasticity])
group_columns = ['source_train_dataset', 'target_train_dataset', 'training_method']
avg_columns = ['test_abs_rel', 'test_sq_rel', 'test_log10', 'test_rmse', 'test_rmse_log', 'test_a1', 'test_a2', 'test_a3']
df_plasticity = df_plasticity.groupby(group_columns, as_index=False)[avg_columns].mean()
#df_plasticity

In [9]:
def compute_stability_plasticity_tradeoff(group):
    product_series = {col: np.prod(group[col]) for col in avg_columns}
    product_series_multiplied_by_two = {col: val * 2 for col, val in product_series.items()}
    sum_series = {col: np.sum(group[col]) for col in avg_columns}
    product_series_multiplied_by_two = pd.Series(product_series_multiplied_by_two)
    sum_series = pd.Series(sum_series)
    return product_series_multiplied_by_two / sum_series

In [10]:
df_stability_plasticity = pd.concat([df_stability, df_plasticity])
#df_stability_plasticity

In [11]:
df_stability_plasticity_tradeoff = df_stability_plasticity.groupby(['source_train_dataset','target_train_dataset','training_method'], as_index=False).apply(compute_stability_plasticity_tradeoff)
df_stability_plasticity_tradeoff

,source_train_dataset,target_train_dataset,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
0,DDAD,KITTI,BOFedSCDepth,0.167446,3.017929,0.082000,13.519774,0.282750,0.742249,0.895749,0.945500
1,DDAD,KITTI,BOFedSCDepth(ConstrainedLoss),0.170906,3.116640,0.083488,13.721497,0.287944,0.736743,0.894000,0.943496
2,DDAD,KITTI,BOFedSCDepth(ConstrainedLossRetrain),0.165498,2.956407,0.080231,13.320681,0.278956,0.748206,0.898486,0.947496
3,DDAD,KITTI,BOFedSCDepth(Retrain),0.166498,2.979677,0.080731,13.351669,0.280223,0.747706,0.898734,0.946998
4,DDAD,KITTI,FedSCDepth,0.197107,3.595584,0.101005,14.139419,0.346512,0.699079,0.842511,0.892304
5,DDAD,KITTI,FedSCDepth(Retrain),0.197806,3.542418,0.101950,14.070890,0.346651,0.695926,0.842879,0.892746
6,DDAD,KITTI,SCDepth,0.152387,2.373314,0.069830,11.441594,0.248203,0.775159,0.916318,0.959250
7,KITTI,DDAD,BOFedSCDepth,0.154578,2.319916,0.068562,11.434071,0.243032,0.790009,0.923331,0.966231
8,KITTI,DDAD,BOFedSCDepth(ConstrainedLoss),0.159498,2.299060,0.068996,11.196094,0.241897,0.784624,0.923240,0.966963
9,KITTI,DDAD,BOFedSCDepth(ConstrainedLossRetrain),0.160488,2.352260,0.070310,11.389430,0.245088,0.778930,0.922014,0.965469


In [12]:
df_stability_ciclic = df_stability.groupby(['training_method'], as_index=False)[avg_columns].mean()
#df_stability_ciclic

In [13]:
df_plasticity_ciclic = df_plasticity.groupby(['training_method'], as_index=False)[avg_columns].mean()
#df_plasticity_ciclic

In [14]:
df_stability_plasticity_ciclic = pd.concat([df_stability_ciclic, df_plasticity_ciclic])
#df_stability_plasticity_ciclic

In [18]:
df_stability_plasticity_ciclic_tradeoff = df_stability_plasticity_ciclic.groupby(['training_method'], as_index=False).apply(compute_stability_plasticity_tradeoff)
df_stability_plasticity_ciclic_tradeoff.sort_values(by=['test_abs_rel'])

,training_method,test_abs_rel,test_sq_rel,test_log10,test_rmse,test_rmse_log,test_a1,test_a2,test_a3
6,SCDepth,0.155161,2.404690,0.067857,11.173358,0.243779,0.782723,0.917292,0.962189
0,BOFedSCDepth,0.161470,2.685425,0.075400,12.488835,0.263209,0.766355,0.909587,0.955870
3,BOFedSCDepth(Retrain),0.161995,2.652182,0.075300,12.372837,0.262326,0.766659,0.911735,0.956999
2,BOFedSCDepth(ConstrainedLossRetrain),0.163839,2.683916,0.075516,12.376192,0.262608,0.764095,0.910340,0.956497
1,BOFedSCDepth(ConstrainedLoss),0.165889,2.728070,0.076403,12.471185,0.265276,0.761026,0.908680,0.955233
5,FedSCDepth(Retrain),0.181427,3.072410,0.089653,13.156329,0.306855,0.737355,0.881631,0.928244
4,FedSCDepth,0.183153,3.118499,0.088943,12.966048,0.305546,0.742807,0.882381,0.928986
